In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 2
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC50(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC50(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC50(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


D:\Hu_code\ResNet18_ESC50\ESC50_kFold_5s\..\TrainDataPreprocessing.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 130.72 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #學習率
    lr= 0.0032
    #epoch
    epoch= 90
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_5S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
Train Epoch: 1/90 Train_Loss: 3.4050729274749756 Train_acc: 10.062500%
Train Epoch: 2/90 Train_Loss: 2.864435911178589 Train_acc: 18.437500%
Train Epoch: 3/90 Train_Loss: 2.4043378829956055 Train_acc: 28.625000%
Train Epoch: 4/90 Train_Loss: 2.0951027870178223 Train_acc: 36.750000%
Train Epoch: 5/90 Train_Loss: 1.6887603998184204 Train_acc: 44.250000%
Train Epoch: 6/90 Train_Loss: 1.2170360088348389 Train_acc: 51.250000%
Train Epoch: 7/90 Train_Loss: 1.4123196601867676 Train_acc: 57.687500%
Train Epoch: 8/90 Train_Loss: 1.3239860534667969 Train_acc: 64.125000%
Train Epoch: 9/90 Train_Loss: 1.0210120677947998 Train_acc: 66.062500%
Train Epoch: 10/90 Train_Loss: 0.8346057534217834 Train_acc: 69.062500%
Train Epoch: 11/90 Train_Loss: 0.87746864

Train Epoch: 13/90 Train_Loss: 0.6333728432655334 Train_acc: 83.937500%
Train Epoch: 14/90 Train_Loss: 0.2749609649181366 Train_acc: 87.625000%
Train Epoch: 15/90 Train_Loss: 0.20699861645698547 Train_acc: 89.687500%
Train Epoch: 16/90 Train_Loss: 0.211883082985878 Train_acc: 93.125000%
Train Epoch: 17/90 Train_Loss: 0.18612945079803467 Train_acc: 96.437500%
Train Epoch: 18/90 Train_Loss: 0.06335513293743134 Train_acc: 97.062500%
Train Epoch: 19/90 Train_Loss: 0.053742416203022 Train_acc: 99.187500%
Train Epoch: 20/90 Train_Loss: 0.03220289945602417 Train_acc: 99.500000%
Train Epoch: 21/90 Train_Loss: 0.042703498154878616 Train_acc: 99.812500%
Train Epoch: 22/90 Train_Loss: 0.024817025288939476 Train_acc: 99.312500%
Train Epoch: 23/90 Train_Loss: 0.025368085131049156 Train_acc: 99.500000%
Train Epoch: 24/90 Train_Loss: 0.10960494726896286 Train_acc: 99.375000%
Train Epoch: 25/90 Train_Loss: 0.026642370969057083 Train_acc: 99.625000%
Train Epoch: 26/90 Train_Loss: 0.025613587349653244 T

Train Epoch: 29/90 Train_Loss: 0.027389628812670708 Train_acc: 97.812500%
Train Epoch: 30/90 Train_Loss: 0.04444315284490585 Train_acc: 98.812500%
Train Epoch: 31/90 Train_Loss: 0.016784638166427612 Train_acc: 99.750000%
Train Epoch: 32/90 Train_Loss: 0.008051195181906223 Train_acc: 100.000000%
Train Epoch: 33/90 Train_Loss: 0.00562567263841629 Train_acc: 100.000000%
Train Epoch: 34/90 Train_Loss: 0.01752026379108429 Train_acc: 99.937500%
Train Epoch: 35/90 Train_Loss: 0.006626973394304514 Train_acc: 99.937500%
Train Epoch: 36/90 Train_Loss: 0.005266061518341303 Train_acc: 100.000000%
Train Epoch: 37/90 Train_Loss: 0.021873632445931435 Train_acc: 100.000000%
Train Epoch: 38/90 Train_Loss: 0.0031420295126736164 Train_acc: 100.000000%
Train Epoch: 39/90 Train_Loss: 0.005908273626118898 Train_acc: 100.000000%
Train Epoch: 40/90 Train_Loss: 0.003598773153498769 Train_acc: 100.000000%
Train Epoch: 41/90 Train_Loss: 0.0038103710394352674 Train_acc: 100.000000%
Train Epoch: 42/90 Train_Loss: 

Train Epoch: 44/90 Train_Loss: 0.0028853341937065125 Train_acc: 100.000000%
Train Epoch: 45/90 Train_Loss: 0.002649440197274089 Train_acc: 100.000000%
Train Epoch: 46/90 Train_Loss: 0.0021307424176484346 Train_acc: 100.000000%
Train Epoch: 47/90 Train_Loss: 0.002446683356538415 Train_acc: 100.000000%
Train Epoch: 48/90 Train_Loss: 0.0018250658176839352 Train_acc: 100.000000%
Train Epoch: 49/90 Train_Loss: 0.0021985138300806284 Train_acc: 100.000000%
Train Epoch: 50/90 Train_Loss: 0.009391255676746368 Train_acc: 100.000000%
Train Epoch: 51/90 Train_Loss: 0.005201998166739941 Train_acc: 99.937500%
Train Epoch: 52/90 Train_Loss: 0.0017087359447032213 Train_acc: 100.000000%
Train Epoch: 53/90 Train_Loss: 0.003079078160226345 Train_acc: 100.000000%
Train Epoch: 54/90 Train_Loss: 0.0030789149459451437 Train_acc: 100.000000%
Train Epoch: 55/90 Train_Loss: 0.0048698000609874725 Train_acc: 100.000000%
Train Epoch: 56/90 Train_Loss: 0.0018559469608590007 Train_acc: 100.000000%
Train Epoch: 57/90

Train Epoch: 59/90 Train_Loss: 0.0016394853591918945 Train_acc: 100.000000%
Train Epoch: 60/90 Train_Loss: 0.01699022576212883 Train_acc: 100.000000%
Train Epoch: 61/90 Train_Loss: 0.0019185603596270084 Train_acc: 100.000000%
Train Epoch: 62/90 Train_Loss: 0.0012475295225158334 Train_acc: 100.000000%
Train Epoch: 63/90 Train_Loss: 0.002236677100881934 Train_acc: 100.000000%
Train Epoch: 64/90 Train_Loss: 0.0019929041154682636 Train_acc: 100.000000%
Train Epoch: 65/90 Train_Loss: 0.0026159638073295355 Train_acc: 100.000000%
Train Epoch: 66/90 Train_Loss: 0.0027595008723437786 Train_acc: 100.000000%
Train Epoch: 67/90 Train_Loss: 0.0026673241518437862 Train_acc: 100.000000%
Train Epoch: 68/90 Train_Loss: 0.0020037146750837564 Train_acc: 100.000000%
Train Epoch: 69/90 Train_Loss: 0.001419986947439611 Train_acc: 100.000000%
Train Epoch: 70/90 Train_Loss: 0.003662261413410306 Train_acc: 100.000000%
Train Epoch: 71/90 Train_Loss: 0.0016883877106010914 Train_acc: 100.000000%
Train Epoch: 72/9

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()